En la siguiente celda realizaremos las importaciones necesarias:

In [4]:
import pickle
import time
from tqdm import tqdm
from getpass import getpass
import selenium.common.exceptions
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.relative_locator import RelativeBy
from selenium.webdriver.support import expected_conditions as EC

Definimos a continuación la clase Tests y sus métodos, de los cuales funciones como la de esperar_elemento se utilizarán tanto en la obtención de elementos de la página como una vez se haya entrenado e el modelo y se utilice para la realización de pruebas [1]

In [12]:
class Tests:
    def __init__(self, info_pantalla):
        # Variables de la clase que almacenarán parámetros de inicio de sesión, dimensiones de la pantalla con mayor resolución y valores con los que realizar comprobaciones
        self.info_pantalla = info_pantalla
        self.usuario = None
        self.contrasenia = None
        self.valores_guardados = {}

    def configurar_driver(self, headless=False):
        # Se configura para que el navegador no notifique acerca de contraseña débil
        self.chrome_options = webdriver.ChromeOptions()
        prefs = {
            'credentials_enable_service': False,
            'profile.password_manager_enabled': False,          
            'profile.password_manager_leak_detection': False
        }

        self.chrome_options.add_experimental_option("prefs", prefs)

        # En caso de que el usuario lo haya indicado, se establece en modo --headless, lo cual agiliza la ejecución
        if headless:
            self.chrome_options.add_argument('--headless')
            self.chrome_options.add_argument('--disable-blink-features=AutomationControlled')
            
        self.chrome_options.add_argument('start-maximized')

        # Indicamos que Chrome será el navegador a utilizar por el driver de Selenium
        self.driver = webdriver.Chrome(self.chrome_options)

        # En caso de no haberse indicado ejecución headless, se realiza la configuración relativa a la pantalla en que se ejecutarán las pruebas de selenium
        if headless is False:
            # Se abre el navegador en aquella pantalla que tenga mayor resolución. Esto se debe a que el SoftWare está programado para uso en pantalla 4K
            self.driver.set_window_position(self.info_pantalla.x, self.info_pantalla.y)
            self.driver.maximize_window()
    
            self.vars = {}
    
            # Se comprueban las dimensiones de la pantalla sobre la que se probará. Si no coinciden, se calcula el zoom a partir del ancho de la pantalla sobre la que se probará
            ancho_requerido = 3840
            alto_requerido = 2160
    
            if self.info_pantalla.width < ancho_requerido and self.info_pantalla.height < alto_requerido:
                self.driver.get('chrome://settings/')
                zoom = self.info_pantalla.width / ancho_requerido
                self.driver.execute_script(f'chrome.settingsPrivate.setDefaultZoom({zoom});')
                self.driver.back()
            
    def configurar_usuario_y_contrasenia(self):
        # De no haberse indicado el usuario y contraseña, en cuyo caso estarían las variables de la clase inicializadas, se solicitan los datos
        self.usuario = getpass('Introduzca su nombre de usuario: ')
        self.contrasenia = getpass('Introduzca su contraseña: ')

    def ir_a_url(self, url):
        # Se accede a la URL indicada por parámetro si dicha URL es diferente a aquella en que el driver se encuentra
        if url != self.driver.current_url:
            self.driver.get(url)

    def pulsar_tecla(self, params):
        # Función que ejecuta el equivalente a la pulsación de una tecla
        if type(params) is list:
            params = params[0]
        
        match params['tecla'].upper():
            case 'ENTER':
                if 'xpath' in params:
                    self.driver.find_element(By.XPATH, params['xpath']).send_keys(Keys.ENTER)

            case _:
                print('La tecla ' + params['tecla'] + ' no está contemplada')

    # Función que comprueba si se encuentra visible un scroll horizontal
    def comprobar_scroll_horizontal_presente(self):
        self.driver.execute_script('return document.documentElement.scrollWidth>document.documentElement.clientWidth;')

    # Función que comprueba si se encuentra visible un scroll vertical
    def comprobar_scroll_vertical_presente(self):
        self.driver.execute_script('return document.documentElement.scrollHeight>document.documentElement.clientHeight;')

    # Función que comprueba el idioma en que se encuentra configurada la web
    def comprobar_idioma(self):
        idioma = self.driver.find_element(By.XPATH, "//html").get_attribute('lang')

    # Función que ejecuta la acción de click de ratón.
    def hacer_clic(self, params):
        elemento = self.buscar_elemento(params)
        if elemento is not None:
            try:
                # Se intenta hacer clic
                elemento.click()
    
            except (selenium.common.exceptions.ElementNotInteractableException, selenium.common.exceptions.ElementClickInterceptedException):
                # Es posible que por el tipo de elemente se produzca error, en cuyo caso se intentaría la siguiente acción
                self.driver.execute_script('arguments[0].click()', elemento)

    # Función de búsqueda de etiqueta, indicada en los parámetros
    def buscar_elemento(self, params):
        elemento = None
        if type(params) is list:
            params = params[0]
        #print('PARAMS BUSCAR ELEMENTO:', params)
        #print('KEYS PARAMS BUSCAR ELEMENTOS:', params.keys())
        try:
            if 'xpath' in params.keys():
                #print('X_PATH:', params['xpath'])
                elemento = self.driver.find_element(By.XPATH, params['xpath'])

            elif 'type' in params.keys() or 'tag' in params.keys():
                if 'tag' in params.keys():
                    params['type'] = params['tag']
                    params.pop('tag')

                elemento_web = self.driver.find_element(By.TAG_NAME, params['type'])
                params.pop('type')
                if len(params.keys()) > 1:
                    for texto_o_atributo_a_buscar in params.keys():
                        match texto_o_atributo_a_buscar:
                            case 'text':
                                elemento = [elemento_web for elemento_web in elemento if elemento_web.text.upper() == params['text'].upper()]
                                
                            case 'attribute':
                                elemento = [elemento_web for elemento_web in elemento if elemento_web.get_attribute(params['attribute'].split('=')[0]) == params['attribute'].split('=')[1]]
                                
                            case 'id':
                                elemento = [elemento_web for elemento_web in elemento if elemento_web.id == params['id']]
                                
                            case 'class':
                                pass
                                
        except selenium.common.exceptions.NoSuchElementException:
            #print('ENTRA EN NOSUCHELEMENTEXCEPTION')
            elemento = None

        return elemento

    # Función equivalente a la anterior. En este caso devuelve todos los elementos encontrados por xpath u otro tipo de identificación
    def buscar_elementos(self, params):
        elementos = None
        if 'xpath' in params.keys():
            elementos = self.driver.find_elements(By.XPATH, params['xpath'])

        elif 'type' in params.keys() or 'tag' in params.keys():
            if 'tag' in params.keys():
                params['type'] = params['tag']
                params.pop('tag')

            elementos = self.driver.find_elements(By.TAG_NAME, params['type'])
            params.pop('type')
            if len(params.keys()) > 1:
                for texto_o_atributo_a_buscar in params.keys():
                    match texto_o_atributo_a_buscar:
                        case 'text':
                            elementos = [elemento_web for elemento_web in elementos if elemento_web.text.upper() == params['text'].upper()]
                            
                        case 'attribute':
                            elementos = [elemento_web for elemento_web in elementos if elemento_web.get_attribute(params['attribute'].split('=')[0]) == params['attribute'].split('=')[1]]
                            
                        case 'id':
                            elementos = [elemento_web for elemento_web in elementos if elemento_web.id == params['id']]
                            
                        case 'class':
                            pass

        return elementos

    # Función de búsqueda de contenido de un elemento web, concretamente los elementos hijos
    def buscar_elementos_hijos(self, elemento_padre, params):
        elementos = None
        if 'xpath' in params.keys():
            elementos = elemento_padre.find_elements(By.XPATH, params['xpath'])

        elif 'type' in params.keys() or 'tag' in params.keys():
            if 'tag' in params.keys():
                params['type'] = params['tag']
                params.pop('tag')

            elementos = self.driver.find_elements(By.TAG_NAME, params['type'])
            params.pop('type')
            if len(params.keys()) > 1:
                for texto_o_atributo_a_buscar in params.keys():
                    match texto_o_atributo_a_buscar:
                        case 'text':
                            elementos = [elemento_web for elemento_web in elemento if elemento_web.text.upper() == params['text'].upper()]

                        case 'attribute':
                            elementos = [elemento_web for elemento_web in elemento if elemento_web.get_attribute(params['attribute'].split('=')[0]) == params['attribute'].split('=')[1]]
                            
                        case 'id':
                            elementos = [elemento_web for elemento_web in elemento if elemento_web.id == params['id']]
                            
                        case 'class':
                            pass

        return elementos

    # Función idéntica a la anterior, si bien en este caso se busca un único elemento
    def buscar_elemento_hijo(self, elemento_padre, params):
        if type(params) is list:
            params = params[0]
        elemento = None
        if 'xpath' in params.keys():
            xpath = params['xpath']
            if xpath.startswith('./*') is False:
                xpath = './*' + xpath

            elemento = elemento_padre.find_element(By.XPATH, params['xpath'])

        elif 'type' in params.keys() or 'tag' in params.keys():
            if 'tag' in params.keys():
                params['type'] = params['tag']
                params.pop('tag')

            elemento = self.driver.find_element(By.TAG_NAME, params['type'])
            params.pop('type')
            if len(params.keys()) > 1:
                for texto_o_atributo_a_buscar in params.keys():
                    match texto_o_atributo_a_buscar:
                        case 'text':
                            elemento = [elemento_web for elemento_web in elemento if elemento_web.text.upper() == params['text'].upper()]
                            
                        case 'attribute':
                            elemento = [elemento_web for elemento_web in elemento if elemento_web.get_attribute(params['attribute'].split('=')[0]) == params['attribute'].split('=')[1]]
                            
                        case 'id':
                            elemento = [elemento_web for elemento_web in elemento if elemento_web.id == params['id']]
                            
                        case 'class':
                            pass

        return elemento

    # Función para selección de elemento, por ejemplo, de una lista con elementos seleccionables
    def seleccionar_elemento(self, params, valor_seleccion):
        try:
            elemento = self.buscar_elemento(params=params)[0]
            element_select = Select(elemento)
            element_select.select_by_value(valor_seleccion)

        except selenium.common.exceptions.UnexpectedTagNameException as e:
            print(e.args)

    def obtener_versión_driver(self, min_version=None):
        if min_version is None:
            return self.driver.capabilities['version']

        else:
            return self.driver.capabilities['version'] >= min_version

    def parar_driver(self):
        self.driver.quit()

    # Función que determina si un elemento se encuentra visible o no
    def comprobar_visibilidad_elemento(self, params):
        match params['tipo']:
            case 'xpath':
                match params['visible']:
                    case True:
                        WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.visibility_of_element_located((By.XPATH, params['esperar']['xpath'])))

                    case False:
                        WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.invisibility_of_element_located((By.XPATH, params['esperar']['xpath'])))

                    case _:
                        print('El tipo de visibilidad no coincide con uno de los posibles: true o false')

            case 'class':
                match params['visible']:
                    case True:
                        WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.visibility_of_element_located((By.CLASS_NAME, params['tipo']['class'])))
                        
                    case False:
                        WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.invisibility_of_element_located((By.CLASS_NAME, params['tipo']['class'])))

                    case _:
                        print('El tipo de visibilidad no coincide con uno de los posibles: true o false')

            case 'id':
                match params['visible']:
                    case True:
                        WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.visibility_of_element_located((By.ID, params['tipo']['id'])))

                    case False:
                        WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.invisibility_of_element_located((By.ID, params['tipo']['id'])))

                    case _:
                        print('El tipo de visibilidad no coincide con uno de los posibles: true o false')


    # Función diseñada especialmente para acciones de tipo hover o colocación de ratón sobre un elemento para que se visibilice un texto emergente
    def esperar_elemento(self, params):
        if type(params) is list:
            params = params[0]
        #print('PARAMS ESPERAR:', params)
        parametro_espera = list(params['esperar'].keys())
        match parametro_espera[0]:
            case 'xpath':
                match params['tipo_espera']:
                    case 'presencia':
                        WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.presence_of_element_located((By.XPATH, params['esperar']['xpath'])))

                    case 'visibilidad':
                        WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.visibility_of_element_located((By.XPATH, params['esperar']['xpath'])))

                    case 'invisibilidad':
                        WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.invisibility_of_element_located((By.XPATH, params['esperar']['xpath'])))

                    case _:
                        print('El tipo de espera no coincide con uno de los posibles: presencia, visibilidad o invisibilidad')

                #WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.visibility_of_element_located((By.XPATH, params['esperar']['xpath'])))

            case 'class':
                WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.presence_of_element_located((By.CLASS_NAME, params['esperar']['class'])))

            case 'id':
                WebDriverWait(self.driver, timeout=100, poll_frequency=1).until(EC.presence_of_element_located((By.ID, params['esperar']['id'])))

    # Función que comprueba la resolución de pantalla
    def comprobar_resolucion_pantalla(self,params):
        if params['ancho'] <= self.info_pantalla.width and params['alto'] <= self.info_pantalla.height:
            print('La pantalla sobre la que se está ejecutando la herramienta tiene al menos la resolución requerida')

        else:
            print('Alguna de las dimensiones no se corresponde\nResolución requerida:', str(params['ancho']) + 'x' + str(params['alto']), '\nResolución encontrada: ' + str(self.info_pantalla.width) + 'x' + str(self.info_pantalla.height))

    # Función que comprueba la versión del driver o navegador web
    def comprobar_version_driver(self, params):
        match list(params.keys())[0]:
            case 'min_version':
                print(int(self.driver.capabilities['browserVersion'].split('.')[0]) >= params['min_version'])

            case 'exact_version':
                print(int(self.driver.capabilities['browserVersion'].split('.')[0]) == params['exact_version'])

    # Función específica del software a comprobar, pues es posible acceder a distintas páginas según elección al inicio de sesión
    def ir_a_centro(self, url, params):
        # Se comprueba si estamos con sesión iniciada y en el centro deseado
        if self.driver.current_url is not None and self.driver.current_url == url:
            body = self.driver.find_element(By.XPATH, '//body')
            if params['centro'] not in body.get_attribute('innerHTML'):
                self.cerrar_sesion()
                self.iniciar_sesion(params)
                
        # Se comprueba si la sesión no está iniciada, en cuyo caso procedemos a iniciar sesión
        else:
            self.iniciar_sesion(params)

    # Función equivalente a la acción de pulsar F11 o poner/quitar pantalla completa
    def poner_quitar_pantalla_completa(self, params):
        veces = 1
        if 'veces' in params.keys():
            veces = int(params['veces'])

        i = 0
        while i < veces:
            body = self.driver.find_element(By.XPATH, '/html/body')
            body.send_keys(webdriver.Keys.F11)

            if veces > 1:
                time.sleep(5)

            i = i + 1

    # Función que comprueba el contenido de un elemento web
    def comprobar_contenido_elemento(self, params):
        comprobacion_ok = True
        elemento_encontrado = self.buscar_elemento(params)
        if 'texto' in params.keys() and params['texto'] not in elemento_encontrado.text:
            comprobacion_ok = False

        elif 'regex' in params.keys() and re.search(params['regex'], elemento_encontrado.text) is not None:
            comprobacion_ok = False

        elif 'atributo' in params.keys():
            valor_atributo = elemento_encontrado.get_attribute(params['atributo']['nombre'])
            if valor_atributo != params['atributo']['valor']:
                comprobacion_ok = False

        assert comprobacion_ok is True, 'OK'

    # Función con la que obtener la fecha y hora en el formato deseado, de interés para comprobaciones como la última actualización de un elemento
    def obtener_fecha_y_hora(self, params):
        fecha_y_hora = time.strftime(params['formato'])
        return fecha_hora

    # Función que comprueba valores relacionados con el estilo, como puede ser el color en que se presenta un texto
    def comprobar_valor_css(self, params):
        comprobacion_ok = False
        elemento = self.buscar_elemento(params)
        valor_css = elemento.value_of_css_property(property_name=params['valor_css'])

        if valor_css == params['valor_comprobacion']:
            comprobacion_ok = True

        assert comprobacion_ok is True, 'OK'

    # Función que comprueba el contenido cuando se coloca el ratón sobre un elemento
    def comprobar_contenido_hover(self, params):
        try:
            #print(params)
            if type(params) is list:
                params = params[0]
                
            # Se busca el elemento sobre el que hacer hover y ponemos el ratón sobre el
            self.hacer_hover(params)

            # Se espera a que esté el elemento deseado presente y después se busca, pues su contenido es el que queremos comprobar
            self.esperar_elemento(params)
            elemento_padre = self.buscar_elemento(params['padre'])
            #print('ELEM PADRE:', elemento_padre)
            # Se determina si contiene lo esperado. Puede ser que deba contener todo lo que se dice o basta con que se cumpla uno entre varios
            comprobacion_ok = None
            match params['tipo_comprobacion'].upper():
                case 'Y' | 'AND':
                    comprobacion_ok = True
                    for params_hijo in params['comprobar']:
                        if type(params_hijo) is list:
                            params_hijo = params_hijo[0]
                        #print(params_hijo)
                        elemento_encontrado = self.buscar_elemento_hijo(elemento_padre, params_hijo)

                        if elemento_encontrado is not None:
                            if 'texto' in params_hijo and params_hijo['texto'] not in elemento_encontrado.text:
                                comprobacion_ok = False

                            elif 'regex' in params_hijo and re.search(params_hijo['regex'], elemento_encontrado.text) is not None:
                                comprobacion_ok = False

                        else:
                            comprobacion_ok = False


                case 'O' | 'OR':
                    comprobacion_ok = False
                    for params_hijo in params['comprobar']:
                        if type(params_hijo) is list:
                            params_hijo = params_hijo[0]
                        #print(params_hijo)
                        elemento_encontrado = self.buscar_elemento_hijo(elemento_padre, params_hijo)
                        if elemento_encontrado is not None:
                            comprobacion_ok = True

                            if 'texto' in params_hijo and params_hijo['texto'] not in elemento_encontrado.text:
                                comprobacion_ok = False

                            elif 'regex' in params_hijo and re.search(params_hijo['regex'], elemento_encontrado.text) is not None:
                                comprobacion_ok = False

                        if comprobacion_ok:
                            break

            #print('Comprobacion_ok', comprobacion_ok)
            assert comprobacion_ok is True, 'OK'

        except selenium.common.exceptions.NoSuchElementException as e:
            print('Fallo por elemento no encontrado')
            raise AssertionError

        except selenium.common.exceptions.TimeoutException as e:
            print('Se ha producido un fallo por timeout')
            raise AssertionError

        l = 0

    # Función para relleno de campos de entrada, como el usuario para el inicio de sesión
    def rellenar_campo(self, params):
        elemento = self.buscar_elemento(params)
        #print('ELEM RELLENAR CAMPO:', elemento)
        if type(params) is list:
            params = params[0]
        
        if type(params['valor']) is dict and 'xpath' not in params['valor'].keys():
            elemento.send_keys(params['valor'])

        else:
            #print('VALOR A RELLENO:', params['valor'])
            elemento.send_keys(str(params['valor']))

    # Función que comprueba el contenido de todos los elementos cuyo xpath, id u otro es el mismo
    def comprobar_contenido_elementos(self, params):
        try:
            # Se buscan los elementos sobre el que comprobar_contenido, que contendrán la misma clase o algún otro parámetro en común
            elementos = self.buscar_elementos(params)

            for elemento in elementos:
                elemento_web = self.buscar_elemento(elemento)
                # Se comprueba si contiene lo esperado. Puede ser que deba contener la totalidad de lo que se dice o basta con que se cumpla uno entre varios
                comprobacion_ok = None
                match params['tipo_comprobacion'].upper():
                    case 'Y' | 'AND':
                        comprobacion_ok = True
                        for params_hijo in params['contenido']:
                            elemento_encontrado = self.buscar_elemento_hijo(elemento_padre, params_hijo)

                            if elemento_encontrado is not None:
                                if 'texto' in params_hijo and params_hijo['texto'] not in elemento_encontrado.text:
                                    comprobacion_ok = False

                                elif 'regex' in params_hijo and re.search(params_hijo['regex'], elemento_encontrado.text) is not None:
                                    comprobacion_ok = False

                            else:
                                comprobacion_ok = False

                    case 'O' | 'OR':
                        comprobacion_ok = False
                        for params_hijo in params['contenido']:
                            elemento_encontrado = self.buscar_elemento_hijo(elemento_padre, params_hijo)
                            if elemento_encontrado is not None:
                                comprobacion_ok = True

                                if 'texto' in params_hijo and params_hijo['texto'] not in elemento_encontrado.text:
                                    comprobacion_ok = False

                                elif 'regex' in params_hijo and re.search(params_hijo['regex'], elemento_encontrado.text) is not None:
                                    comprobacion_ok = False

                            if comprobacion_ok:
                                break

                #print('Comprobacion_ok', comprobacion_ok)
                assert comprobacion_ok is True, 'OK'

        except selenium.common.exceptions.NoSuchElementException as e:
            print('Fallo por elemento no encontrado')
            raise AssertionError

        except selenium.common.exceptions.TimeoutException as e:
            print('Fallo por tiempo máximo excedido')
            raise AssertionError

        l = 0

    
    def comprobar_contenido_elemento_2(self, params):
        try:
            # Se buscan los elementos sobre el que comprobar_contenido, que contendrán la misma clase o algún otro parámetro en común
            elemento_padre = self.buscar_elemento(params)

            if type(params) is list:
                params = params[0]
                
            comprobacion_ok = None
            match params['tipo_comprobacion'].upper():
                case 'Y' | 'AND':
                    comprobacion_ok = True
                    for params_hijo in params['contenido']:
                        if type(params_hijo) is list:
                            params_hijo = params_hijo[0]
                        
                        elemento_encontrado = self.buscar_elemento_hijo(elemento_padre, params_hijo)

                        if elemento_encontrado is not None:
                            if 'texto' in params_hijo and params_hijo['texto'] not in elemento_encontrado.text:
                                comprobacion_ok = False

                            elif 'regex' in params_hijo and re.search(params_hijo['regex'], elemento_encontrado.text) is not None:
                                comprobacion_ok = False

                        else:
                            comprobacion_ok = False

                case 'O' | 'OR':
                    comprobacion_ok = False
                    for params_hijo in params['contenido']:
                        if type(params_hijo) is list:
                            params_hijo = params_hijo[0]
                        
                        elemento_encontrado = self.buscar_elemento_hijo(elemento_padre, params_hijo)
                        if elemento_encontrado is not None:
                            comprobacion_ok = True

                            if 'texto' in params_hijo and params_hijo['texto'] not in elemento_encontrado.text:
                                comprobacion_ok = False

                            elif 'regex' in params_hijo and re.search(params_hijo['regex'], elemento_encontrado.text) is not None:
                                comprobacion_ok = False

                        if comprobacion_ok:
                            break

            #print('Comprobacion_ok', comprobacion_ok)
            assert comprobacion_ok is True, 'OK'

        except selenium.common.exceptions.NoSuchElementException as e:
            print('Fallo por elemento no encontrado')
            raise AssertionError

        except selenium.common.exceptions.TimeoutException as e:
            print('Fallo por timeout')
            raise AssertionError

        l = 0

    # Se comprueba la URL en la que el driver se encuentra actualmente
    def comprobar_url_actual(self, params):
        print(params['url_comprobacion'] == self.driver.current_url)

    # Función de comprobación del contenido de un diálogo, como puede ser la confirmación de una acción o información acerca de un error
    def comprobar_contenido_dialogo(self, params):
        try:
            # Se buscan el elemento sobre el que comprobar_contenido
            elemento = self.buscar_elemento(params)

            #time.sleep(1)
            # Se espera a que esté el elemento deseado presente y después lo Se buscan, pues su contenido es el que queremos comprobar
            self.esperar_elemento(params)
            elemento_padre = self.buscar_elemento(params['esperar'])

            # Se comprueba si contiene lo esperado. Puede ser que deba contener la totalidad de lo que se dice o basta con que se cumpla uno entre varios
            comprobacion_ok = None
            match params['tipo_comprobacion'].upper():
                case 'Y' | 'AND':
                    comprobacion_ok = True
                    for params_hijo in params['contenido']:
                        elemento_encontrado = self.buscar_elemento_hijo(elemento_padre, params_hijo)

                        if elemento_encontrado is not None:
                            if 'texto' in params_hijo and params_hijo['texto'] not in elemento_encontrado.text:
                                comprobacion_ok = False

                            elif 'regex' in params_hijo and re.search(params_hijo['regex'], elemento_encontrado.text) is not None:
                                comprobacion_ok = False

                        else:
                            comprobacion_ok = False

                case 'O' | 'OR':
                    comprobacion_ok = False
                    for params_hijo in params['contenido']:
                        elemento_encontrado = self.buscar_elemento_hijo(elemento_padre, params_hijo)
                        if elemento_encontrado is not None:
                            comprobacion_ok = True

                            if 'texto' in params_hijo and params_hijo['texto'] not in elemento_encontrado.text:
                                comprobacion_ok = False

                            elif 'regex' in params_hijo and re.search(params_hijo['regex'], elemento_encontrado.text) is not None:
                                comprobacion_ok = False

                        if comprobacion_ok:
                            break

            #print('Comprobacion_ok', comprobacion_ok)
            assert comprobacion_ok is True, 'OK'

        except selenium.common.exceptions.NoSuchElementException as e:
            #print('Error selenium.common.exceptions.NoSuchElementException')
            raise AssertionError

        except selenium.common.exceptions.TimeoutException as e:
            #print('Error selenium.common.exceptions.TimeoutException')
            raise AssertionError

        l = 0

    # Función para acciones de tipo drag&drop
    def mover_elemento(self, params):
        elements = self.driver.find_elements(By.XPATH, '//div[@class="tab-ws"]')
        element_drag = WebDriverWait(self.driver, 60).until(
            EC.element_to_be_clickable(elements[0]))
        ActionChains(self.driver).drag_and_drop_by_offset(element_drag, 50, 0).perform()

    # Función que comprueba que es posible iniciar sesión correctamente
    def comprobar_inicio_sesion(self, params):
        if self.usuario is None:
            self.configurar_usuario_y_contrasenia()

        url_acceso = getpass('Indique la URL a la que se debe acceder:' )
        self.driver.get(url_acceso)
        time.sleep(5)

        login_type = params['login_type']
        match login_type:
            case 'inicio_sesion_correcto':
                # Introducimos nombre de usuario
                self.driver.find_element(By.ID, "input-16").send_keys(self.usuario)

                # Introducimos contraseña
                self.driver.find_element(By.ID, "password").send_keys(self.contrasenia)

                # Hacemos clic en el botón de login
                self.driver.find_element(By.CSS_SELECTOR, ".mr-3 > strong").click()
                self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.F11)
                time.sleep(3)

            case 'campos_vacios':
                # Usuario y contraseña vacíos
                self.driver.find_element(By.ID, "input-16").send_keys('')
                self.driver.find_element(By.ID, "password").send_keys('')
                self.driver.find_element(By.CSS_SELECTOR, ".mr-3 > strong").click()
                time.sleep(3)

            case 'sin_contrasenia':
                # Usuario correcto pero sin contraseña
                self.driver.find_element(By.ID, "input-16").send_keys(self.usuario)
                self.driver.find_element(By.ID, "password").send_keys('')
                self.driver.find_element(By.CSS_SELECTOR, ".mr-3 > strong").click()
                time.sleep(3)

            case 'sin_usuario':
                # Contraseña correcta pero sin usuario
                self.driver.find_element(By.ID, "input-16").send_keys('')
                self.driver.find_element(By.ID, "password").send_keys(self.contrasenia)
                self.driver.find_element(By.CSS_SELECTOR, ".mr-3 > strong").click()
                time.sleep(3)

            case 'usuario_contrasenia_erroneos':
                # User vacío y contrasenia correcto
                self.driver.find_element(By.ID, "input-16").send_keys('aaaaaaaaaaaaaaaaaaa')
                self.driver.find_element(By.ID, "password").send_keys('bbbbbbbbbb')
                self.driver.find_element(By.CSS_SELECTOR, ".mr-3 > strong").click()
                time.sleep(3)
                i = 0

        if 'centro' in params.keys():
            params = {'xpath': '//span[@class="dl-ui-select__label dl-ui-select__inputtext"]'}
            self.hacer_clic(params)
            time.sleep(5)
            match params['centro']:
                case 'GCCC' | 'Canarias' | 'CANARIAS':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="GCCC"]').click()

                case 'LECB' | 'Barcelona' | 'BARCELONA':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="LECB"]').click()

                case 'LECM' | 'Madrid' | 'MADRID':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="LECM"]').click()

                case 'LECP' | 'Palma' | 'PALMA':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="LECP"]').click()

                case 'LECS' | 'Sevilla' | 'SEVILLA':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="LECS"]').click()

            self.driver.find_element(By.CSS_SELECTOR, ".mr-3 > strong").click()
            time.sleep(2)
            self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.F11)

        time.sleep(30)

    # Función que valida un cierre de sesión correcto
    def comprobar_cierre_sesion(self):
        # Se espera a que se pueda presionar el botón y que exista
        try:
            elemento = WebDriverWait(self.driver, 30).until(EC.presence_of_element_located((By.XPATH, '//div[@id="user"]//*/span/*[local-name()="svg"]/*[local-name()="path"]')))
            elements = self.driver.find_elements(By.XPATH, '//div[@id="user"]//*/span/*[local-name()="svg"]/*[local-name()="path"]')
            elements[-1].click()
            
        except selenium.common.TimeoutException:
            print('No se ha conseguido acceder al botón de cierre de sesión en 30seg')
            return None

        try:
            self.esperar_elemento({'esperar': {'xpath': '//div[@id="app"]/div/div/div/footer/button'}, 'tipo_espera': 'presencia'})
            self.hacer_clic({'xpath': '//div[@id="app"]/div/div/div/footer/button'})

        except selenium.common.TimeoutException:
            print('No se ha conseguido acceder al botón de confirmación de cierre de sesión en 30seg')
            return None

    # Función para inicio de sesión en el sistema. Se trata de una función específica para la herramienta a validar
    def iniciar_sesion(self, params):
        if self.usuario is None:
            self.configurar_usuario_y_contrasenia()

        url_acceso = getpass('Indique la URL a la que se debe acceder:' )
        self.driver.get(url_acceso)
        time.sleep(5)
        
        # Introducimos nombre de usuario
        self.driver.find_element(By.ID, "input-16").send_keys(self.usuario)

        time.sleep(5)
        
        # Introducimos contraseña
        self.driver.find_element(By.ID, "password").send_keys(self.contrasenia)

        time.sleep(5)
        
        # Hacemos clic en el botón de login
        self.driver.find_element(By.CSS_SELECTOR, ".mr-3 > strong").click()
        #self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.F11)
        time.sleep(3)

        if 'centro' in params.keys():
            self.hacer_clic({'xpath': '//span[@class="dl-ui-select__label dl-ui-select__inputtext"]'})
            time.sleep(3)
            match params['centro']:
                case 'GCCC' | 'Canarias' | 'CANARIAS':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="GCCC"]').click()

                case 'LECB' | 'Barcelona' | 'BARCELONA':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="LECB"]').click()

                case 'LECM' | 'Madrid' | 'MADRID':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="LECM"]').click()

                case 'LECP' | 'Palma' | 'PALMA':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="LECP"]').click()

                case 'LECS' | 'Sevilla' | 'SEVILLA':
                    self.driver.find_element(By.XPATH, '//li[@aria-label="LECS"]').click()

            time.sleep(5)
            self.driver.find_element(By.CSS_SELECTOR, ".mr-3 > strong").click()


    # Cierre de sesión o salida de la aplicación
    def cerrar_sesion(self):
        # Se espera a que se pueda presionar el botón y que exista
        try:
            elemento = WebDriverWait(self.driver, 30).until(EC.presence_of_element_located((By.XPATH, '//div[@id="user"]//*/span/*[local-name()="svg"]/*[local-name()="path"]')))
            elements = self.driver.find_elements(By.XPATH, '//div[@id="user"]//*/span/*[local-name()="svg"]/*[local-name()="path"]')
            elements[-1].click()
        except selenium.common.TimeoutException:
            print('No se ha conseguido acceder al botón de cierre de sesión en 30seg')
            return None

        try:
            self.esperar_elemento({'esperar': {'xpath': '//div[@id="app"]/div/div/div/footer/button'}, 'tipo_espera': 'presencia'})
            self.hacer_clic({'xpath': '//div[@id="app"]/div/div/div/footer/button'})
            #elemento = WebDriverWait(self.driver, 30).until(EC.presence_of_element_located((By.XPATH, '//div[@id="app"]/div/div/div/footer/button')))
            #self.hacer_clic(elemento)

        except selenium.common.TimeoutException:
            print('No se ha conseguido acceder al botón de confirmación de cierre de sesión en 30seg')
            return None

        #elemento = self.driver.find_element(By.XPATH, '//div[@id="app"]/div/div/div/footer/button')
        #self.hacer_clic(elemento)

    # Función que realiza la acción equivalente a colocar el ratón sobre un elemento
    def hacer_hover(self, params):
        elemento = self.buscar_elemento(params)
        ActionChains(self.driver).move_to_element(elemento).perform()

    # Función que realiza la misma acción que realizar un clic con el botón derecho del ratón
    def hacer_clic_derecho(self, params):
        elemento = self.buscar_elemento(params)
        ActionChains(self.driver).context_click(elemento).perform()

    # Función que comprueba si un elemento se encuentra habilitado, como un botón
    def comprobar_elemento_habilitado(self, params):
        elemento = self.buscar_elemento(params)

        if type(params) is list:
            params = params[0]
            
        if params['habilitado']:
            #print(elemento.is_enabled())
            assert elemento.is_enabled(), 'OK'

        else:
            #print(elemento.is_enabled() is False)
            assert elemento.is_enabled() is False, 'OK'

    # Función idéntica a la anterior, pero para la comprobación de todos aquellos elementos con etiqueta idéntica
    def comprobar_elementos_habilitados(self, params):
        elementos = self.buscar_elementos(params)
        for elemento in elementos:
            if params['habilitado']:
                #print(elemento.is_enabled())
                assert elemento.is_enabled(), 'OK'

            else:
                #print(elemento.is_enabled() is False)
                assert elemento.is_enabled() is False, 'OK'

    def comprobar_existencia_elemento(self, params):
        comprobacion_ok = False

        elemento = self.buscar_elemento(params)
        if type(params) is list:
            params = params[0]
        if (params['existe'] and elemento is not None) or (params['existe'] is False and elemento is None):
            comprobacion_ok = True

        assert comprobacion_ok is True, 'OK'

    # Función a utilizar en el futuro, para calcular datos como la diferencia temporal
    def calcular_diferencia(self, params):
        pass

    # Función que comprueba la disposición de elementos
    def comprobar_posicion_relativa_elementos(self, params):
        encontrado = None
        match params['posicion']:
            case 'encima':
                encontrado = RelativeBy('xpath', params['elemento_1']['xpath']).above(self.buscar_elemento(params['elemento_2']))

            case 'debajo':
                encontrado = RelativeBy('xpath', params['elemento_1']['xpath']).below(self.buscar_elemento(params['elemento_2']))

            case 'a la izquierda':
                encontrado = RelativeBy('xpath', params['elemento_1']['xpath']).toLeftOf(self.buscar_elemento(params['elemento_2']))

            case 'a la derecha':
                encontrado = RelativeBy('xpath', params['elemento_1']['xpath']).toRightOf(self.buscar_elemento(params['elemento_2']))

            case 'cerca':
                # Estar cerca es estar ambos a una distancia máxima de 50px
                encontrado = RelativeBy('xpath', params['elemento_1']['xpath']).near(self.buscar_elemento(params['elemento_2']))

        #print('Comprobacion ok', encontrado is not None)
        assert encontrado is not None, 'OK'

    # Función para la comparación de elementos, como si dos elementos son iguales o uno es mayor que otro
    def comparar_elementos(self, params):
        if type(params) is list:
            params = params[0]
        #print('PARAMS comparar_elementos:', params)

        comprobacion_ok = False
        valor_elemento_1 = None
        valor_elemento_2 = None
        elemento_1 = self.buscar_elemento(params['elemento_1'])
        if elemento_1 is not None:
            if 'atributo' in params['elemento_1'].keys():
                valor_elemento_1 = elemento_1.get_attribute(params['elemento_1']['atributo'])

            elif 'valor_guardado' in params['elemento_1'].keys():
                valor_elemento_1 = self.valores_guardados[params['elemento_1']['id']]

            else:
                valor_elemento_1 = len(self.buscar_elementos(params['elemento_1']))

        elemento_2 = self.buscar_elemento(params['elemento_2'])
        if elemento_2 is not None:
            if 'atributo' in params['elemento_2'].keys():
                valor_elemento_2 = elemento_2.get_attribute(params['elemento_2']['atributo'])

            elif 'valor_guardado' in params['elemento_2'].keys():
                valor_elemento_2 = self.valores_guardados[params['elemento_2']['id']]

            else:
                valor_elemento_2 = len(self.buscar_elementos(params['elemento_2']))

        match params['tipo_comprobacion']:
            case 'igual':
                comprobacion_ok = valor_elemento_1 == valor_elemento_2

            case 'distinto' | 'distintos':
                comprobacion_ok = valor_elemento_1 != valor_elemento_2

            case 'menor':
                comprobacion_ok = valor_elemento_1 < valor_elemento_2

            case 'mayor':
                comprobacion_ok = valor_elemento_1 > valor_elemento_2

            case 'menor o igual':
                comprobacion_ok = valor_elemento_1 <= valor_elemento_2

            case 'mayor o igual':
                comprobacion_ok = valor_elemento_1 >= valor_elemento_2
                
        assert comprobacion_ok is True, 'OK'

    # Función para la comprobación de número de elementos con misma etiqueta, como por ejemplo el número de escritorios
    def comprobar_cantidad_elementos(self, params):
        comprobacion_ok = False
        if type(params) is list:
            params = params[0]
            
        elementos = self.buscar_elementos(params)
        match params['tipo_comprobacion']:
            case 'obtener':
                self.valores_guardados[params['id']] = len(elementos)

            case 'igual':
                comprobacion_ok = len(elementos) == params['cantidad']

            case 'menor':
                comprobacion_ok = len(elementos) < params['cantidad']

            case 'mayor':
                comprobacion_ok = len(elementos) > params['cantidad']

            case 'menor o igual':
                comprobacion_ok = len(elementos) <= params['cantidad']

            case 'mayor o igual':
                comprobacion_ok = len(elementos) <= params['cantidad']

        assert comprobacion_ok is True, 'OK'

    # Función que comprueba si la serie de elementos indicados por parámetro existen
    def comprobar_existencia_elementos(self, params):
        elementos_a_comprobar = params['comprobar']
        comprobacion_ok = None
        match params['tipo_comprobacion'].upper():
            case 'Y' | 'AND':
                comprobacion_ok = True
                i = 0
                while i < len(elementos_a_comprobar) and comprobacion_ok:
                    elemento = self.buscar_elemento(elementos_a_comprobar[i])
                    if (params['comprobar_existe'] and elemento is None) or (params['comprobar_existe'] is False and elemento is not None):
                        comprobacion_ok = False

                    i = i + 1

            case 'O' | 'OR':
                comprobacion_ok = False
                i = 0
                while i < len(elementos_a_comprobar):
                    elemento = self.buscar_elemento(elementos_a_comprobar[i])
                    if (params['comprobar_existe'] and elemento is not None) or (params['comprobar_existe'] is False and elemento is None):
                        comprobacion_ok = True
                        break

                    i = i + 1

        assert comprobacion_ok is True, 'OK'

    # Función a completar a futuro, que será de utilidad para asegurar que se parte de unas condiciones en que podrá ejecutarse la validación requerida
    def hacer_si_no_existe(self, params):
        if 'buscar' in params.keys():
            elemento = self.buscar_elemento(params['buscar'])
            if elemento is None:
                for accion in params['hacer']:
                    pass

        else:
            pass

    # Función que guarda un valor o una serie de valores obtenidos. De utilidad para comprobación de cambios o uso para determinadas pruebas
    def guardar_valor(self, params):
        elemento = self.buscar_elemento(params)
        #print('ELEM:',elemento)
        #print('PARAMS:',params)

        if type(params) is list:
            params = params[0]
        
        match params['valor_a_guardar']:
            case 'texto':
                self.valores_guardados[params['id']] = elemento.text

            case 'regex':
                texto = re.search(params['regex'], elemento.text)
                if texto is not None:
                    self.valores_guardados[params['id']] = texto.group(0)

            case 'html':
                self.valores_guardados[params['id']] = self.buscar_elemento(params).get_attribute('outerHTML')

**<h1>Referencias bibliográficas:**</h1>
1. *Selenium with Python — Selenium Python Bindings 2 documentation*. (s.f.). https://selenium-python.readthedocs.io/